In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [40]:
path_train = "C:/Users/Amministratore/.jupyter/Weather Data-20251115/train.csv"
path_test = "C:/Users/Amministratore/.jupyter/Weather Data-20251115/test.csv"
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

print("Shape train:", df_train.shape)
print("Shape test:", df_test.shape)

Shape train: (7579, 92)
Shape test: (3247, 90)


In [41]:
df_train.head()

,fkl010h0_ANT,fkl010h0_BAS,fkl010h0_DAV,fkl010h0_DOL,fkl010h0_GVE,fkl010h0_INT,fkl010h0_LUG,fkl010h0_SIO,fkl010h0_STG,fkl010h0_ZER,...,ure200h0_SIO,ure200h0_STG,ure200h0_ZER,tre200h0,tre200h0_lag24h,target_tre200h0_plus12h,target_tre200h0_plus24h,target_tre200h0_plus48h,hour,season
0,1.3,2.1,0.8,8.3,1.0,0.6,0.6,0.5,3.4,0.9,...,84.4,76.0,74.0,12.2,8.5,17.5,10.3,10.0,5.0,Spring
1,3.8,0.8,3.8,14.0,6.9,0.5,2.9,0.6,2.0,1.8,...,87.6,96.3,67.4,4.3,8.9,12.1,1.5,6.3,3.0,Spring
2,1.1,2.0,0.1,10.5,2.1,1.3,1.1,1.7,0.9,1.1,...,81.9,96.5,65.8,-12.3,-7.6,-7.1,-3.5,-2.1,0.0,Winter
3,0.8,3.1,1.8,9.8,1.0,1.9,0.8,0.8,0.5,0.8,...,46.4,83.9,33.7,5.6,8.2,3.2,7.4,6.5,13.0,Autumn
4,4.4,1.6,4.3,2.9,2.2,2.8,6.3,5.4,1.0,2.8,...,39.4,65.7,44.7,20.7,19.2,15.7,21.9,23.9,11.0,Summer


In [42]:
df_test = df_test.iloc[:, 1:] 
df_test.head()

,fkl010h0_ANT,fkl010h0_BAS,fkl010h0_DAV,fkl010h0_DOL,fkl010h0_GVE,fkl010h0_INT,fkl010h0_LUG,fkl010h0_SIO,fkl010h0_STG,fkl010h0_ZER,...,ure200h0_GVE,ure200h0_INT,ure200h0_LUG,ure200h0_SIO,ure200h0_STG,ure200h0_ZER,tre200h0,tre200h0_lag24h,hour,season
0,8.1,1.7,4.5,13.5,6.8,2.2,0.6,0.6,4.5,4.8,...,87.1,96.2,62.5,92.6,85.7,72.9,5.5,5.9,13,Winter
1,0.9,1.8,1.3,3.1,0.9,1.6,0.8,0.6,0.8,1.1,...,79.3,76.4,59.4,69.4,77.4,33.7,1.9,5.5,18,Winter
2,2.9,2.1,0.8,5.8,0.7,0.5,1.1,0.9,4.3,1.0,...,72.7,94.3,80.1,84.2,84.5,74.5,17.7,17.5,0,Spring
3,5.4,2.6,3.2,5.7,5.2,2.3,0.5,8.2,1.7,1.7,...,46.1,54.7,48.4,35.6,60.2,34.8,7.5,4.4,16,Spring
4,0.9,0.6,1.1,1.7,0.5,0.7,0.5,0.5,0.6,0.9,...,86.5,85.9,81.7,74.3,64.2,65.6,11.7,14.6,1,Spring


In [45]:
df_train = df_train.dropna()
df_test = df_test.fillna(df_test.mean(numeric_only=True))

for col in df_test.select_dtypes(include=["object"]).columns:
    df_test[col] = df_test[col].fillna(df_test[col].mode()[0])

targets = ["target_tre200h0_plus12h","target_tre200h0_plus24h","target_tre200h0_plus48h"]
features = [c for c in df_train.columns if c not in targets] 

num_cols = [c for c in features if df_train[c].dtype != "object"]
cat_cols = [c for c in features if df_train[c].dtype == "object"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

def train_and_predict(target_name):
    print(f"\nTraining modello Linear Regression → {target_name}")

    X_train = df_train[features]
    y_train = df_train[target_name]

    model = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("reg", LinearRegression())
    ])

    model.fit(X_train, y_train)

    preds = model.predict(df_test[features])

    return preds.flatten()  # <-- garantisce 1-D array

preds_12 = train_and_predict("target_tre200h0_plus12h")
preds_24 = train_and_predict("target_tre200h0_plus24h")
preds_48 = train_and_predict("target_tre200h0_plus48h")

results = pd.DataFrame({
    "target_tre200h0_plus12h": preds_12,
    "target_tre200h0_plus24h": preds_24,
    "target_tre200h0_plus48h": preds_48
})


Training modello Linear Regression → target_tre200h0_plus12h

Training modello Linear Regression → target_tre200h0_plus24h

Training modello Linear Regression → target_tre200h0_plus48h


In [47]:
submission_24 = pd.DataFrame({
    "Id": range(1, len(preds_24) + 1),
    "target_tre200h0_plus24h": preds_24
})

# Salva file CSV
submission_24.to_csv("submission_plus24h.csv", index=False)